In [ ]:
import pandas as pd
import copy

In [ ]:
xls = pd.ExcelFile('Cricket Data Set.xlsx')
xls

In [ ]:
print(len(xls.sheet_names))
xls.sheet_names

In [ ]:
df_names = ['_'.join(sheet_names.lower().split()) + '_df' for sheet_names in xls.sheet_names]
print(len(df_names))
df_names

In [ ]:
df_dict = {df_names[sheet_index] : pd.read_excel(xls,sheet_index) for sheet_index in range(len(df_names))}
df_dict.keys()

In [ ]:
for df_name in df_names:
    print(df_name)
    display(df_dict[df_name].head(5))

In [ ]:
df_dict['control_sheet_df'].head(30)

In [ ]:
df_dict['control_sheet_df'].rename({'Unnamed: 0': 'Shot Type'}, axis=1, inplace=True)
df_dict['control_sheet_df'].drop('Unnamed: 1',axis = 1, inplace = True)

In [ ]:
df_dict['control_sheet_df'].head(30)

In [ ]:
df_dict['control_sheet_df'].shape

In [ ]:
for row_index in range(8,df_dict['control_sheet_df'].shape[0],9):
    print(row_index)

In [ ]:
df_dict['control_sheet_df'].isna().sum()

In [ ]:
df_dict['control_sheet_df'] = df_dict['control_sheet_df'].dropna(how='all') 
df_dict['control_sheet_df'].isna().sum()

In [ ]:
df_dict['parameters_df']

In [ ]:
shot_type = df_dict['control_sheet_df'].pop('Shot Type')

In [ ]:
shot_type

In [ ]:
shot_type.value_counts()

In [ ]:
shot_type.isna().sum()

In [ ]:
shot_type_temp = copy.deepcopy(shot_type)
shot_type_temp

In [ ]:
for index,value in shot_type_temp.items():
    if type(value) != str:
        shot_type_temp[index] = shot_type_temp[index-1] 
shot_type_temp

In [ ]:
shot_type_temp.value_counts()

In [ ]:
shot_type = shot_type_temp
shot_type.value_counts()

In [ ]:
df_dict['control_sheet_df'].shape

In [ ]:
ball_type_df = df_dict['parameters_df'].loc[:,['Ball Type','ball symbol']].dropna()
ball_type_dict = dict(zip(ball_type_df['ball symbol'],ball_type_df['Ball Type']))
ball_type_dict

In [ ]:
print(df_dict['control_sheet_df'].columns)
df_dict['control_sheet_df'].rename(columns = {'Sl':'SL'}, inplace = True)
df_dict['control_sheet_df'].columns

In [ ]:
def generate_ballwise_df(df = df_dict['control_sheet_df']):
    for col_index in range(0,df.shape[1],3):
        ball_symbol = df.iloc[:,col_index:col_index+3].columns[0]
        ball_type = ball_type_dict[ball_symbol]
    
        df_dict[f'{ball_type}_shots_df'] = df.iloc[:,col_index:col_index+3]
        df_dict[f'{ball_type}_shots_df'].columns = ['Runs Scored', 'Correct execution of shot according to the ball', 'Effectiveness of the shot execution']
        df_dict[f'{ball_type}_shots_df'].insert(0, 'Ball Type', ball_symbol)
        df_dict[f'{ball_type}_shots_df'].insert(1,'Shot Type', shot_type)

In [ ]:
generate_ballwise_df()

In [ ]:
for df_name in df_dict.keys():
    print(df_name)
    display(df_dict[df_name].head(10))

In [ ]:
df_dict['analysis_parameters_df'] = df_dict['analysis_parameters_df'].dropna(how = 'all')
df_dict['analysis_parameters_df'].reset_index(drop = True, inplace = True)

In [ ]:
pd.set_option('display.max_colwidth', None)
df_dict['analysis_parameters_df']

In [ ]:
def unique_values(df):
    unique_vals = [(col,list(df[col].unique())) for col in df.columns if col!='Sno']
    return unique_vals

In [ ]:
for df_name in df_dict.keys():
    if 'vs' not in df_name:
        continue 
    print(df_name)
    for col,vals in unique_values(df_dict[df_name]):
        print(f'{col} : {vals}')
    print()

In [ ]:
for df_name in df_dict.keys():
    if 'vs' in df_name:
        continue 
    print(df_name)
    for col,vals in unique_values(df_dict[df_name]):
        print(f'{col} : {vals}')
    print()

In [ ]:
for df_name in df_dict.keys():
    if 'vs' not in df_name:
        continue
    print(df_name)
    print(df_dict[df_name][df_dict[df_name]['Dismissal kind'].notnull()])

In [ ]:
df_dict['ind_vs_afg_df']['Dismissal kind'] = df_dict['ind_vs_afg_df']['Dismissal kind'].replace('Leg Stump', 'B')
df_dict['ind_vs_afg_df']['Dismissal kind'] = df_dict['ind_vs_afg_df']['Dismissal kind'].replace('Stump Out', 'St')

In [ ]:
df_dict['ind_vs_afg_df'].loc[df_dict['ind_vs_afg_df']['Dismissal kind'].notnull() & df_dict['ind_vs_afg_df']['Wicket'].isna(),'Wicket'] = 'Wicket'

In [ ]:
def clean_missing_values(df):
    cols_nan = [col_name for col_name, nan_count in df.isna().sum().items() if nan_count!=0 ]
    unary_cols = ['Pitch Type', 'Match ID']
    binary_cols = ['Wide', 'No ball', 'Wicket', 'Hit the Bat']
    
    for col in cols_nan:
        if col in unary_cols:
            df.drop(col, axis = 1, inplace = True)
            continue
        elif col in binary_cols:
            df[col] = df[col].notnull().astype('float')
            continue
        
        df[col].fillna(value = 0, inplace = True)
    
    return df

In [ ]:
def drop_incorrect_data(df,column_name,child_values,parent_values):
    if all(value in parent_values for x in child_values):
        return

    incorrect_values = [val for val in child_values if val not in parent_values]
    df.drop(df[df[column_name].isin(incorrect_values)].index, inplace=True)
    return

In [ ]:
def validate_column_values(df):
    ball_symbol_list = df_dict['parameters_df']['ball symbol'].dropna().tolist()
    ball_symbol_list = list(map(lambda x: x.upper(),ball_symbol_list))
    
    shot_symbol_list = df_dict['parameters_df']['shot symbol'].dropna().tolist()
    shot_symbol_list = list(map(lambda x: x.upper(),shot_symbol_list))
    
    dismissal_symbol_list = df_dict['parameters_df']['dismisal symbol'].dropna().tolist()
    dismissal_symbol_list = list(map(lambda x: x.upper(),dismissal_symbol_list))
    dismissal_dict = dict(zip(df_dict['parameters_df']['Dismissal Kind'].dropna(),df_dict['parameters_df']['dismisal symbol'].dropna()))
    
    control_values_list = [0.0,1.0]
    effectiveness_values_list = [x/10.0 for x in range(0,11)] + [-1.0]

    for col_name, col_values in unique_values(df):
        if 'shot' in col_name.lower():
            col_values = list(map(lambda x: x.upper(), col_values))
            drop_incorrect_data(df,col_name,col_values,shot_symbol_list)
        elif col_name.lower() == 'ball type':
            col_values = list(map(lambda x: x.upper(), col_values))
            drop_incorrect_data(df,col_name,col_values,ball_symbol_list)
        elif 'control' in col_name.lower():
            df[col_name] = df[col_name].replace(-1.0,0.0) if -1.0 in col_values else df[col_name]
            col_values.remove(-1.0) if -1.0 in col_values else col_values
            drop_incorrect_data(df,col_name,col_values,control_values_list)
        elif 'effectiveness' in col_name.lower():
            drop_incorrect_data(df,col_name,col_values,effectiveness_values_list)
        elif 'dismissal' in col_name.lower():
            col_values.remove(0)
            df[col_name] = df[col_name].replace('C','Ct') if 'C' in col_values else df[col_name]
            col_values = ['Ct' if col_value == 'C' else col_value for col_value in col_values]
            
            for col_value in col_values:
                df[col_name] = df[col_name].replace(col_value,dismissal_dict[col_value]) if len(col_value) > 3 else df[col_name
                                                                                                                      ]
            col_values = [dismissal_dict[col_value] if len(col_value) > 3 else col_value for col_value in col_values]
            col_values = [col_value.upper() for col_value in col_values]
            drop_incorrect_data(df,col_name,col_values,dismissal_symbol_list)
    
    return df

In [ ]:
def clean_df(df):
    df.drop('Sno', axis = 1, inplace = True)
    df = df.dropna(how = 'all')
    df.reset_index(drop = True, inplace = True)
    df.rename(columns = {'Player Name':'Batsman Name', 'Player ID':'Batsman Player ID', 'Bowler':'Bowler Name', 'Player ID.1':'Bowler Player ID','X':'Effectiveness'}, inplace = True)
    
    non_nan_df = clean_missing_values(df)    
    validated_df = validate_column_values(non_nan_df)
    
    return validated_df

In [ ]:
length = 0

for df_name in df_dict.keys():
    if 'vs' not in df_name:
        continue 
    length += df_dict[df_name].shape[0]

print(length)

In [ ]:
matches_df = pd.DataFrame()

for df_name in df_dict.keys():
    if 'vs' not in df_name:
        continue
    cleaned_df = clean_df(df_dict[df_name])
    matches_df = pd.concat([matches_df, cleaned_df], ignore_index=True)

matches_df.shape

In [ ]:
for df_name in df_dict.keys():
    if 'vs' not in df_name:
        continue
    print(df_name)
    for col_name, col_values in unique_values(df_dict[df_name]):
        print(col_name, col_values)
    print()

In [ ]:
matches_df

In [ ]:
for col_name, col_values in unique_values(matches_df):
    print(col_name,col_values)

In [ ]:
matches_df.nunique()

In [ ]:
player_info = dict(zip(matches_df['Batsman Name'],matches_df['Batsman Player ID']))
player_info

In [ ]:
len(player_info)

In [ ]:
matches_df['Batsman Name'].replace('Kedar Jadhav','Kedhar Jadhav', inplace = True)
matches_df['Batsman Name'].replace('Shikar Dhawan','Shikhar Dhawan', inplace = True)

In [ ]:
matches_df.nunique()

In [ ]:
df_dict['analysis_parameters_df']

In [ ]:
def off_side_score(df = matches_df):
    off_side_shots = ['DR','C','RS']
    
    off_side_shots_count = df[df['Type of Shot'].isin(off_side_shots)].groupby('Batsman Name').agg(off_side_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    off_side_score_df = off_side_shots_count.merge(grouped_df,on = 'Batsman Name')
    off_side_score_df = off_side_score_df.reset_index()
    
    off_side_score_df['Highest percentage of off side shots'] = (off_side_score_df['off_side_shots_count']/off_side_score_df['total_no_of_balls_faced'])*100
    off_side_score_df['Highest effectiveness average of off side shots'] = off_side_score_df['sum_effectiveness']/off_side_score_df['off_side_shots_count']
    off_side_score_df['off_side_score'] = (off_side_score_df['Highest percentage of off side shots']/100) * off_side_score_df['Highest effectiveness average of off side shots']
    
    off_side_score_df = off_side_score_df.sort_values(by=['off_side_score'], ascending=False)
    
    return off_side_score_df.loc[:,['Batsman Name','Highest percentage of off side shots','Highest effectiveness average of off side shots','off_side_score']]

In [ ]:
off_side_score()

In [ ]:
def leg_side_score(df = matches_df):
    leg_side_shots = ['P','ODR','LG','F','S']
    
    leg_side_shots_count = df[df['Type of Shot'].isin(leg_side_shots)].groupby('Batsman Name').agg(leg_side_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    leg_side_score_df = leg_side_shots_count.merge(grouped_df,on = 'Batsman Name')
    leg_side_score_df = leg_side_score_df.reset_index()
    
    leg_side_score_df['Highest percentage of leg side shots'] = (leg_side_score_df['leg_side_shots_count']/leg_side_score_df['total_no_of_balls_faced'])*100
    leg_side_score_df['Highest effectiveness average of leg side shots'] = leg_side_score_df['sum_effectiveness']/leg_side_score_df['leg_side_shots_count']
    leg_side_score_df['leg_side_score'] = (leg_side_score_df['Highest percentage of leg side shots']/100) * leg_side_score_df['Highest effectiveness average of leg side shots']
    
    leg_side_score_df = leg_side_score_df.sort_values(by=['leg_side_score'], ascending=False)
    
    return leg_side_score_df.loc[:,['Batsman Name','Highest percentage of leg side shots','Highest effectiveness average of leg side shots','leg_side_score']]

In [ ]:
leg_side_score()

In [ ]:
def bouncy_track_score(df = matches_df):
    bouncy_track_shots = ['D','P','C','L']
    
    bouncy_track_shots_count = df[df['Type of Shot'].isin(bouncy_track_shots)].groupby('Batsman Name').agg(bouncy_track_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    bouncy_track_score_df = bouncy_track_shots_count.merge(grouped_df,on = 'Batsman Name')
    bouncy_track_score_df = bouncy_track_score_df.reset_index()
    
    bouncy_track_score_df['Highest percentage of bouncy track shots'] = (bouncy_track_score_df['bouncy_track_shots_count']/bouncy_track_score_df['total_no_of_balls_faced'])*100
    bouncy_track_score_df['Highest effectiveness average of bouncy track shots'] = bouncy_track_score_df['sum_effectiveness']/bouncy_track_score_df['bouncy_track_shots_count']
    bouncy_track_score_df['bouncy_track_score'] = (bouncy_track_score_df['Highest percentage of bouncy track shots']/100) * bouncy_track_score_df['Highest effectiveness average of bouncy track shots']
    
    bouncy_track_score_df = bouncy_track_score_df.sort_values(by=['bouncy_track_score'], ascending=False)
    
    return bouncy_track_score_df.loc[:,['Batsman Name','Highest percentage of bouncy track shots','Highest effectiveness average of bouncy track shots','bouncy_track_score']]

In [ ]:
bouncy_track_score()

In [ ]:
def aggressive_shots_score(df = matches_df):
    aggressive_shots = ['Dr', 'C', 'P', 'Sw', 'RS', 'ODR', 'F', 'R', 'LG', 'SC']
    
    aggressive_shots_count = df[df['Type of Shot'].isin(aggressive_shots)].groupby('Batsman Name').agg(aggressive_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    aggressive_score_df = aggressive_shots_count.merge(grouped_df,on = 'Batsman Name')
    aggressive_score_df = aggressive_score_df.reset_index()
    
    aggressive_score_df['Highest percentage of aggressive shots'] = (aggressive_score_df['aggressive_shots_count']/aggressive_score_df['total_no_of_balls_faced'])*100
    aggressive_score_df['Highest effectiveness average of aggressive shots'] = aggressive_score_df['sum_effectiveness']/aggressive_score_df['aggressive_shots_count']
    aggressive_score_df['aggressive_score'] = (aggressive_score_df['Highest percentage of aggressive shots']/100) * aggressive_score_df['Highest effectiveness average of aggressive shots']
    
    aggressive_score_df = aggressive_score_df.sort_values(by=['aggressive_score'], ascending=False)
    
    return aggressive_score_df.loc[:,['Batsman Name','Highest percentage of aggressive shots','Highest effectiveness average of aggressive shots','aggressive_score']]

In [ ]:
aggressive_shots_score()

In [ ]:
def defense_score(df = matches_df):
    defense_shots = ['D','L']
    
    defense_shots_count = df[df['Type of Shot'].isin(defense_shots)].groupby('Batsman Name').agg(defense_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    defense_score_df = defense_shots_count.merge(grouped_df,on = 'Batsman Name')
    defense_score_df = defense_score_df.reset_index()
    
    defense_score_df['Highest percentage of defense shots'] = (defense_score_df['defense_shots_count']/defense_score_df['total_no_of_balls_faced'])*100
    defense_score_df['Highest effectiveness average of defense shots'] = defense_score_df['sum_effectiveness']/defense_score_df['defense_shots_count']
    defense_score_df['defense_score'] = (defense_score_df['Highest percentage of defense shots']/100) * defense_score_df['Highest effectiveness average of defense shots']
    
    defense_score_df = defense_score_df.sort_values(by=['defense_score'], ascending=False)
    
    return defense_score_df.loc[:,['Batsman Name','Highest percentage of defense shots','Highest effectiveness average of defense shots','defense_score']]

In [ ]:
defense_score()

In [ ]:
def spin_ball_score(df = matches_df):
    spin_ball_shots = ['D', 'Dr', 'C', 'P', 'Sw', 'RS', 'ODR', 'F', 'R', 'LG', 'SC']
    spin_ball = ['GY', 'LB']
    
    spin_ball_shots_count = df[df['Type of Shot'].isin(spin_ball_shots) & df['Ball type'].isin(spin_ball)].groupby('Batsman Name').agg(spin_ball_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    spin_ball_score_df = spin_ball_shots_count.merge(grouped_df,on = 'Batsman Name')
    spin_ball_score_df = spin_ball_score_df.reset_index()
    
    spin_ball_score_df['Highest percentage of spin ball shots'] = (spin_ball_score_df['spin_ball_shots_count']/spin_ball_score_df['total_no_of_balls_faced'])*100
    spin_ball_score_df['Highest effectiveness average of spin ball shots'] = spin_ball_score_df['sum_effectiveness']/spin_ball_score_df['spin_ball_shots_count']
    spin_ball_score_df['spin_ball_score'] = (spin_ball_score_df['Highest percentage of spin ball shots']/100) * spin_ball_score_df['Highest effectiveness average of spin ball shots']
    
    spin_ball_score_df = spin_ball_score_df.sort_values(by=['spin_ball_score'], ascending=False)
    
    return spin_ball_score_df.loc[:,['Batsman Name','Highest percentage of spin ball shots','Highest effectiveness average of spin ball shots','spin_ball_score']]

In [ ]:
spin_ball_score()

In [ ]:
def control_score(df = matches_df):
    shots = ['D', 'Dr', 'C', 'P', 'Sw', 'RS', 'ODR', 'F', 'R', 'LG', 'SC', 'L']
    
    shot_count_df = df[df['Control'] == 1.0].groupby('Batsman Name').agg(controlled_shots_count = ('Type of Shot', 'count'), sum_effectiveness = ('Effectiveness', 'sum'))
    grouped_df = df.groupby('Batsman Name').agg(total_no_of_balls_faced = ('Type of Shot', 'count'))
    
    control_score_df = shot_count_df.merge(grouped_df, on ='Batsman Name')
    control_score_df = control_score_df.reset_index()
    
    control_score_df['Highest percentage of controlled shots'] = (control_score_df['controlled_shots_count']/control_score_df['total_no_of_balls_faced'])*100
    control_score_df['Highest effectiveness average of controlled shots'] = control_score_df['sum_effectiveness']/control_score_df['controlled_shots_count']
    control_score_df['control_score'] = (control_score_df['Highest percentage of controlled shots']/100) * control_score_df['Highest effectiveness average of controlled shots']
    
    control_score_df = control_score_df.sort_values(by=['control_score'], ascending=False)
    
    return control_score_df.loc[:,['Batsman Name','Highest percentage of controlled shots','Highest effectiveness average of controlled shots','control_score']]

In [ ]:
control_score()

In [ ]:
matches_df.info()

In [ ]:
matches_df['Control'].unique()

In [ ]:
matches_df['Control'] = matches_df['Control'].astype('float')

In [ ]:
matches_df.info()

In [ ]:
matches_df['Control'].unique()

In [ ]:
ball_type_dict = dict(zip(df_dict['parameters_df']['ball symbol'],df_dict['parameters_df']['Ball Type']))
ball_type_dict

In [ ]:
shot_count_df = matches_df.groupby(['Ball type', 'Type of Shot']).agg(Count = ('Type of Shot','count'))
shot_count_df = shot_count_df.reset_index()
shot_count_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for ball_type in list(matches_df['Ball type'].unique()):
    plt.figure(figsize = (15,5))
    sns.set_style('whitegrid')
    plt.title(f"Shot Type Spread vs {ball_type_dict[ball_type]} ball")
    sns.countplot(x = 'Type of Shot', data = matches_df[matches_df['Ball type'] == ball_type])

In [ ]:
matches_df.columns

In [ ]:
original_matches_df = copy.deepcopy(matches_df)

In [ ]:
original_matches_df

In [ ]:
df_dict['off_side_score_df'] = off_side_score()
df_dict['leg_side_score_df'] = leg_side_score()
df_dict['bouncy_track_score_df'] = bouncy_track_score()
df_dict['aggressive_score_df'] = aggressive_shots_score()
df_dict['defense_score_df'] = defense_score()
df_dict['spin_ball_score_df'] = spin_ball_score()
df_dict['control_score_df'] = control_score()

In [ ]:
for df_name in df_dict.keys():
    if 'score' not in df_name:
        continue
    score_col = df_name[:-3]
    matches_df = pd.merge(matches_df, df_dict[df_name].loc[:,['Batsman Name',score_col]], on = 'Batsman Name')

In [ ]:
matches_df

In [ ]:
categorical_variables = []
continuous_variables = []

for column in matches_df.columns:
    if matches_df[column].dtype != 'object':
        continuous_variables.append(column)
        continue
    categorical_variables.append(column)

print(categorical_variables)
print(continuous_variables)   

In [ ]:
from scipy.stats import chi2_contingency

alpha = 0.05

for col in categorical_variables:
    contingency_table = pd.crosstab(matches_df[col], matches_df['Effectiveness'])
    if contingency_table.empty:
        continue
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    print(f"Column {col}: chi2={chi2}, p_value={p_value}")
    if p_value > alpha:
        print(f"Column {col} is not significantly related to the target and will be removed.")
        matches_df.drop(col, axis=1, inplace=True)
    print()

matches_df

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in matches_df.columns:
    if col not in categorical_variables:
        continue
    matches_df[col] = le.fit_transform(matches_df[col].astype(str))

In [ ]:
matches_df

In [ ]:
for col_name, col_values in unique_values(matches_df):
    print(col_name,col_values)

In [ ]:
corr = matches_df.corr() 

plt.figure(figsize=(10, 10), dpi = 600) 
 
sns.heatmap(corr, cmap="Blues", annot=True, annot_kws={"size": 8})

plt.tick_params(axis = 'x', labelsize = 12) 
plt.tick_params(axis = 'y', labelsize = 12) 

In [ ]:
X = matches_df.drop("Effectiveness", axis=1)
y = matches_df['Effectiveness']

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 
import catboost as cb
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np

reg1 = svm.SVR(kernel = 'rbf')
reg2 = LinearRegression()
reg3 = RandomForestRegressor()
reg4 = XGBRegressor()
reg5 = LinearRegression()
reg6 = GradientBoostingRegressor()
reg7 = DecisionTreeRegressor()
reg8 = cb.CatBoostRegressor()


score1 = cross_val_score(reg1,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score2 = cross_val_score(reg2,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score3 = cross_val_score(reg3,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score4 = cross_val_score(reg4,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score5 = cross_val_score(reg5,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score6 = cross_val_score(reg6,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score7 = cross_val_score(reg7,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')
score8 = cross_val_score(reg8,X_train,y_train,cv = 5, scoring = 'neg_mean_absolute_error')


print('SVM mean absolute error: ',np.mean(-1*score1))
print('Linear Regression mean absolute error: ',np.mean(-1*score2))
print('Random Forest mean absolute error: ',np.mean(-1*score3))
print('XGBoost mean absolute error: ',np.mean(-1*score4))
print('Linear Regression mean absolute error: ',np.mean(-1*score5))
print('GradientBoosting Regressor mean absolute error: ',np.mean(-1*score6))
print('Decision Tree Regressor mean absolute error: ',np.mean(-1*score7))
print('Cat Boost Regressor mean absolute error: ',np.mean(-1*score8))

Learning rate set to 0.041492
0:	learn: 0.3644924	total: 1.18ms	remaining: 1.18s
1:	learn: 0.3562463	total: 2.15ms	remaining: 1.07s
2:	learn: 0.3481300	total: 3.08ms	remaining: 1.02s
3:	learn: 0.3400650	total: 3.81ms	remaining: 950ms
4:	learn: 0.3344886	total: 4.47ms	remaining: 890ms
5:	learn: 0.3274860	total: 5.32ms	remaining: 882ms
6:	learn: 0.3208305	total: 6.24ms	remaining: 885ms
7:	learn: 0.3145653	total: 7.1ms	remaining: 880ms
8:	learn: 0.3087422	total: 7.94ms	remaining: 874ms
9:	learn: 0.3038810	total: 8.78ms	remaining: 869ms
10:	learn: 0.2986821	total: 9.6ms	remaining: 863ms
11:	learn: 0.2932185	total: 10.4ms	remaining: 860ms
12:	learn: 0.2880820	total: 11.3ms	remaining: 857ms
13:	learn: 0.2834210	total: 12.2ms	remaining: 862ms
14:	learn: 0.2792252	total: 13.1ms	remaining: 859ms
15:	learn: 0.2752747	total: 14ms	remaining: 860ms
16:	learn: 0.2719233	total: 14.8ms	remaining: 856ms
17:	learn: 0.2679534	total: 15.7ms	remaining: 856ms
18:	learn: 0.2649640	total: 16.6ms	remaining: 85

191:	learn: 0.1212670	total: 162ms	remaining: 681ms
192:	learn: 0.1202501	total: 163ms	remaining: 681ms
193:	learn: 0.1200714	total: 164ms	remaining: 681ms
194:	learn: 0.1198589	total: 165ms	remaining: 679ms
195:	learn: 0.1195920	total: 165ms	remaining: 679ms
196:	learn: 0.1191968	total: 166ms	remaining: 678ms
197:	learn: 0.1190701	total: 167ms	remaining: 677ms
198:	learn: 0.1188800	total: 168ms	remaining: 676ms
199:	learn: 0.1182769	total: 169ms	remaining: 676ms
200:	learn: 0.1181574	total: 170ms	remaining: 675ms
201:	learn: 0.1180137	total: 171ms	remaining: 674ms
202:	learn: 0.1174778	total: 171ms	remaining: 673ms
203:	learn: 0.1173442	total: 172ms	remaining: 672ms
204:	learn: 0.1166366	total: 173ms	remaining: 671ms
205:	learn: 0.1166263	total: 174ms	remaining: 669ms
206:	learn: 0.1163631	total: 175ms	remaining: 669ms
207:	learn: 0.1153061	total: 175ms	remaining: 668ms
208:	learn: 0.1151863	total: 176ms	remaining: 667ms
209:	learn: 0.1141318	total: 177ms	remaining: 666ms
210:	learn: 

386:	learn: 0.0651001	total: 321ms	remaining: 509ms
387:	learn: 0.0648955	total: 322ms	remaining: 508ms
388:	learn: 0.0648726	total: 323ms	remaining: 507ms
389:	learn: 0.0647550	total: 324ms	remaining: 507ms
390:	learn: 0.0645175	total: 325ms	remaining: 506ms
391:	learn: 0.0644560	total: 326ms	remaining: 505ms
392:	learn: 0.0642954	total: 327ms	remaining: 505ms
393:	learn: 0.0641662	total: 328ms	remaining: 504ms
394:	learn: 0.0637947	total: 328ms	remaining: 503ms
395:	learn: 0.0637622	total: 329ms	remaining: 502ms
396:	learn: 0.0634350	total: 330ms	remaining: 501ms
397:	learn: 0.0634127	total: 331ms	remaining: 500ms
398:	learn: 0.0633139	total: 332ms	remaining: 500ms
399:	learn: 0.0632393	total: 332ms	remaining: 499ms
400:	learn: 0.0630509	total: 333ms	remaining: 498ms
401:	learn: 0.0630118	total: 334ms	remaining: 497ms
402:	learn: 0.0629029	total: 335ms	remaining: 496ms
403:	learn: 0.0627220	total: 336ms	remaining: 496ms
404:	learn: 0.0626861	total: 337ms	remaining: 495ms
405:	learn: 

570:	learn: 0.0484182	total: 473ms	remaining: 355ms
571:	learn: 0.0483562	total: 474ms	remaining: 355ms
572:	learn: 0.0482832	total: 475ms	remaining: 354ms
573:	learn: 0.0482048	total: 476ms	remaining: 353ms
574:	learn: 0.0481274	total: 477ms	remaining: 352ms
575:	learn: 0.0480407	total: 478ms	remaining: 352ms
576:	learn: 0.0479695	total: 479ms	remaining: 351ms
577:	learn: 0.0479108	total: 479ms	remaining: 350ms
578:	learn: 0.0478448	total: 480ms	remaining: 349ms
579:	learn: 0.0477430	total: 481ms	remaining: 348ms
580:	learn: 0.0476695	total: 482ms	remaining: 348ms
581:	learn: 0.0476462	total: 483ms	remaining: 347ms
582:	learn: 0.0476173	total: 484ms	remaining: 346ms
583:	learn: 0.0475187	total: 485ms	remaining: 345ms
584:	learn: 0.0474292	total: 485ms	remaining: 344ms
585:	learn: 0.0473576	total: 486ms	remaining: 344ms
586:	learn: 0.0473471	total: 487ms	remaining: 343ms
587:	learn: 0.0471465	total: 488ms	remaining: 342ms
588:	learn: 0.0470348	total: 489ms	remaining: 341ms
589:	learn: 

769:	learn: 0.0370240	total: 634ms	remaining: 189ms
770:	learn: 0.0370086	total: 634ms	remaining: 188ms
771:	learn: 0.0369201	total: 635ms	remaining: 188ms
772:	learn: 0.0369178	total: 636ms	remaining: 187ms
773:	learn: 0.0368399	total: 637ms	remaining: 186ms
774:	learn: 0.0367502	total: 638ms	remaining: 185ms
775:	learn: 0.0367139	total: 639ms	remaining: 184ms
776:	learn: 0.0367064	total: 640ms	remaining: 184ms
777:	learn: 0.0367043	total: 641ms	remaining: 183ms
778:	learn: 0.0366486	total: 641ms	remaining: 182ms
779:	learn: 0.0365520	total: 642ms	remaining: 181ms
780:	learn: 0.0364822	total: 643ms	remaining: 180ms
781:	learn: 0.0364254	total: 644ms	remaining: 180ms
782:	learn: 0.0364231	total: 645ms	remaining: 179ms
783:	learn: 0.0363881	total: 646ms	remaining: 178ms
784:	learn: 0.0363860	total: 646ms	remaining: 177ms
785:	learn: 0.0362944	total: 647ms	remaining: 176ms
786:	learn: 0.0362746	total: 648ms	remaining: 175ms
787:	learn: 0.0362579	total: 649ms	remaining: 175ms
788:	learn: 

...

219:	learn: 0.1016542	total: 204ms	remaining: 723ms
220:	learn: 0.1012888	total: 205ms	remaining: 723ms
221:	learn: 0.1011228	total: 206ms	remaining: 722ms
222:	learn: 0.1005864	total: 207ms	remaining: 721ms
223:	learn: 0.1003245	total: 208ms	remaining: 720ms
224:	learn: 0.1002046	total: 209ms	remaining: 719ms
225:	learn: 0.0999594	total: 209ms	remaining: 717ms
226:	learn: 0.0994887	total: 210ms	remaining: 716ms
227:	learn: 0.0993179	total: 211ms	remaining: 715ms
228:	learn: 0.0993116	total: 212ms	remaining: 713ms
229:	learn: 0.0990281	total: 213ms	remaining: 712ms
230:	learn: 0.0985471	total: 213ms	remaining: 710ms
231:	learn: 0.0984276	total: 214ms	remaining: 709ms
232:	learn: 0.0982127	total: 215ms	remaining: 708ms
233:	learn: 0.0980509	total: 216ms	remaining: 707ms
234:	learn: 0.0979178	total: 217ms	remaining: 706ms
235:	learn: 0.0973869	total: 218ms	remaining: 705ms
236:	learn: 0.0971586	total: 219ms	remaining: 704ms
237:	learn: 0.0970204	total: 220ms	remaining: 703ms
238:	learn: 

398:	learn: 0.0618108	total: 361ms	remaining: 543ms
399:	learn: 0.0616744	total: 362ms	remaining: 543ms
400:	learn: 0.0615772	total: 363ms	remaining: 542ms
401:	learn: 0.0614963	total: 363ms	remaining: 541ms
402:	learn: 0.0613988	total: 365ms	remaining: 540ms
403:	learn: 0.0612676	total: 365ms	remaining: 539ms
404:	learn: 0.0611924	total: 366ms	remaining: 538ms
405:	learn: 0.0610919	total: 367ms	remaining: 537ms
406:	learn: 0.0610238	total: 368ms	remaining: 536ms
407:	learn: 0.0608666	total: 369ms	remaining: 535ms
408:	learn: 0.0607348	total: 370ms	remaining: 534ms
409:	learn: 0.0605029	total: 371ms	remaining: 533ms
410:	learn: 0.0604046	total: 371ms	remaining: 532ms
411:	learn: 0.0601301	total: 372ms	remaining: 531ms
412:	learn: 0.0598923	total: 373ms	remaining: 530ms
413:	learn: 0.0598201	total: 374ms	remaining: 530ms
414:	learn: 0.0597406	total: 375ms	remaining: 529ms
415:	learn: 0.0596630	total: 376ms	remaining: 528ms
416:	learn: 0.0595875	total: 377ms	remaining: 527ms
417:	learn: 

598:	learn: 0.0417350	total: 524ms	remaining: 351ms
599:	learn: 0.0416609	total: 525ms	remaining: 350ms
600:	learn: 0.0416013	total: 526ms	remaining: 349ms
601:	learn: 0.0415477	total: 527ms	remaining: 348ms
602:	learn: 0.0413957	total: 527ms	remaining: 347ms
603:	learn: 0.0413390	total: 528ms	remaining: 346ms
604:	learn: 0.0412442	total: 529ms	remaining: 345ms
605:	learn: 0.0411817	total: 530ms	remaining: 345ms
606:	learn: 0.0410990	total: 531ms	remaining: 344ms
607:	learn: 0.0410031	total: 532ms	remaining: 343ms
608:	learn: 0.0409648	total: 532ms	remaining: 342ms
609:	learn: 0.0409097	total: 533ms	remaining: 341ms
610:	learn: 0.0407933	total: 534ms	remaining: 340ms
611:	learn: 0.0407633	total: 535ms	remaining: 339ms
612:	learn: 0.0406317	total: 536ms	remaining: 338ms
613:	learn: 0.0405740	total: 537ms	remaining: 337ms
614:	learn: 0.0404556	total: 538ms	remaining: 337ms
615:	learn: 0.0403482	total: 539ms	remaining: 336ms
616:	learn: 0.0402640	total: 539ms	remaining: 335ms
617:	learn: 

804:	learn: 0.0296170	total: 689ms	remaining: 167ms
805:	learn: 0.0295897	total: 690ms	remaining: 166ms
806:	learn: 0.0295289	total: 691ms	remaining: 165ms
807:	learn: 0.0294722	total: 692ms	remaining: 164ms
808:	learn: 0.0293848	total: 693ms	remaining: 164ms
809:	learn: 0.0293450	total: 694ms	remaining: 163ms
810:	learn: 0.0293196	total: 694ms	remaining: 162ms
811:	learn: 0.0292967	total: 695ms	remaining: 161ms
812:	learn: 0.0292614	total: 696ms	remaining: 160ms
813:	learn: 0.0292205	total: 697ms	remaining: 159ms
814:	learn: 0.0291444	total: 698ms	remaining: 158ms
815:	learn: 0.0291207	total: 699ms	remaining: 158ms
816:	learn: 0.0291037	total: 699ms	remaining: 157ms
817:	learn: 0.0290751	total: 700ms	remaining: 156ms
818:	learn: 0.0290446	total: 701ms	remaining: 155ms
819:	learn: 0.0290297	total: 702ms	remaining: 154ms
820:	learn: 0.0289521	total: 703ms	remaining: 153ms
821:	learn: 0.0289188	total: 704ms	remaining: 152ms
822:	learn: 0.0288777	total: 705ms	remaining: 152ms
823:	learn: 

SVM mean absolute error:  0.21556857131932508
Linear Regression mean absolute error:  31574708513.996727
Random Forest mean absolute error:  0.04142647058823548
XGBoost mean absolute error:  0.04478300510618069
Linear Regression mean absolute error:  31574708513.996727
GradientBoosting Regressor mean absolute error:  0.07065985749796871
Decision Tree Regressor mean absolute error:  0.029264705882353144
Cat Boost Regressor mean absolute error:  0.04878898138180454


In [ ]:
def fit_and_evaluate(reg, param) -> list:
    reg.set_params(**param)
    reg.fit(X_train, y_train)

    y_predict = reg.predict(X_test)
    r2 = r2_score(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)
    mse = mean_squared_error(y_test, y_predict)
    rmse = mean_squared_error(y_test, y_predict, squared=False)
        
    return [reg.__class__.__name__, r2, mae, mse, rmse, param.keys(), param.values()]

In [ ]:
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score)
import random 
import sys

RANDOM_SEED = 12345
random.seed(RANDOM_SEED)

list_grid = []

reg = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-3)
print(reg.__class__.__name__)
for max_depth in np.arange(3, 16, 1):
        sys.stdout.write(f"\rmax_depth: {max_depth}/15...")
        for n_estimators in np.arange(5, 20, 1):
                param = {"max_depth": max_depth,
                        "n_estimators": n_estimators}
                list_grid.append(fit_and_evaluate(reg, param))

                
reg = XGBRegressor(random_state=RANDOM_SEED, n_jobs=-3, verbosity=0)
print()
print(reg.__class__.__name__)
for max_depth in np.arange(3, 11, 1):
        sys.stdout.write(f"\rmax_depth: {max_depth}/10...")
        for n_estimators in [10, 20, 50, 100]:
                for learning_rate in [0.1, 0.2, 0.3]:
                        for subsample in np.arange(0.4, 1.01, 0.1):
                                for colsample_bytree in np.arange(0.4, 1.01, 0.1): 
                                        param = {"max_depth": max_depth,
                                                "n_estimators": n_estimators,
                                                "learning_rate": learning_rate,
                                                "subsample": subsample, 
                                                "colsample_bytree": colsample_bytree}
                                        list_grid.append(fit_and_evaluate(reg, param))  

In [ ]:
reg_df = pd.DataFrame(data=list_grid, columns=["reg", "R2", "MAE", "MSE", "RMSE", "param.keys", "param.values"], index=None)
reg_df.sort_values(by=["R2"], ascending=False, inplace=True)

In [ ]:
reg_df[reg_df['reg'] == "RandomForestRegressor"].head(10)

In [ ]:
reg_df[reg_df['reg'] == "XGBRegressor"].head(10)

In [ ]:
xgb_df = reg_df[reg_df['reg'] == "XGBRegressor"].head(10)
xgb_df['param.keys'] = xgb_df['param.keys'].apply(lambda x: list(x))
xgb_df['param.values'] = xgb_df['param.values'].apply(lambda x: list(x))

parameters_list = []
for keys,values in zip(xgb_df['param.keys'],xgb_df['param.values']):
    reg = XGBRegressor(random_state=RANDOM_SEED, n_jobs=-3, verbosity=0, **dict(zip(keys,values)))
    reg.fit(X_train, y_train) 
    y_pred = reg.predict(X_test)
    print(dict(zip(keys,values)))
    print(r2_score(y_test, y_pred))
    print()

In [ ]:
reg = XGBRegressor(random_state=RANDOM_SEED, n_jobs=-3, verbosity=0, max_depth=9, n_estimators=100, learning_rate=0.2, subsample=0.6, colsample_bytree=0.9999999999999999)
reg.fit(X_train, y_train) 
y_pred = reg.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
top_features = pd.Series(reg.feature_importances_, index = matches_df.drop('Effectiveness',axis = 1).columns).sort_values()
top_features.plot(kind="barh", figsize=(15, 10), title="Top Features")
plt.show()

In [ ]:
rf_df = reg_df[reg_df['reg'] == "RandomForestRegressor"].head(10)
rf_df['param.keys'] = rf_df['param.keys'].apply(lambda x: list(x))
rf_df['param.values'] = rf_df['param.values'].apply(lambda x: list(x))

parameters_list = []
for keys,values in zip(rf_df['param.keys'],rf_df['param.values']):
    reg = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-3, **dict(zip(keys,values)))
    reg.fit(X_train, y_train) 
    y_pred = reg.predict(X_test)
    print(dict(zip(keys,values)))
    print(r2_score(y_test, y_pred))
    print()

In [ ]:
reg = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs = -3, max_depth = 8, n_estimators = 9)
reg.fit(X_train, y_train) 
y_pred = reg.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
top_features = pd.Series(reg.feature_importances_, index = matches_df.drop('Effectiveness',axis = 1).columns).sort_values()
top_features.plot(kind="barh", figsize=(15, 10), title="Top Features")
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

In [ ]:
reg = DecisionTreeRegressor(random_state=RANDOM_SEED, **best_params)
reg.fit(X_train, y_train) 
y_pred = reg.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
top_features = pd.Series(reg.feature_importances_, index = matches_df.drop('Effectiveness',axis = 1).columns).sort_values()
top_features.plot(kind="barh", figsize=(15, 10), title="Top Features")
plt.show()

In [ ]:
parameters = {'depth' : [6,8,10],'learning_rate' : [0.01, 0.05, 0.1], 'iterations' : [30, 50, 100]}
grid = GridSearchCV(estimator = cb.CatBoostRegressor(), param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
print("\n The best score across ALL searched params:\n", grid.best_score_)
print("\n The best parameters across ALL searched params:\n", grid.best_params_)

In [ ]:
reg = cb.CatBoostRegressor(random_state=RANDOM_SEED, **grid.best_params_)
reg.fit(X_train, y_train) 
y_pred = reg.predict(X_test)

r2_score(y_test, y_pred)

In [ ]:
top_features = pd.Series(reg.feature_importances_, index = matches_df.drop('Effectiveness',axis = 1).columns).sort_values()
top_features.plot(kind="barh", figsize=(15, 10), title="Top Features")
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

matches_df.to_csv('matches_df.csv')

dataframe = pd.read_csv("matches_df.csv", header=None)
dataframe = dataframe[1:]
dataset = dataframe.values

X = dataset[:,[i for i in range(25) if i!=13]]
Y = dataset[:,13]

def baseline_model():
    model = Sequential()
    model.add(Dense(24, input_shape=(24,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(model=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
from sklearn.pipeline import Pipeline
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, Y, cv=kfold, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
dataframe = pd.read_csv("matches_df.csv", header=None)
dataframe = dataframe[1:]
dataset = dataframe.values

X = dataset[:,[i for i in range(25) if i!=13]]
y = dataset[:,13]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from keras.optimizers import Adam
import keras
from keras.callbacks import EarlyStopping
import pandas as pd 

model = Sequential()
model.add(Dense(128, activation="relu", input_dim=6))
model.add(Dense(32, activation="relu"))
model.add(Dense(8, activation="relu"))

model.add(Dense(1, activation="linear"))

model.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10000000, batch_size=100, verbose=2, callbacks=[es])

PredTestSet = model.predict(X1)
PredValSet = model.predict(X2)

numpy.savetxt("trainresults.csv", PredTestSet, delimiter=",")
numpy.savetxt("valresults.csv", PredValSet, delimiter=",")